In [ ]:
import os
import sys

path = os.getcwd()
while os.path.basename(os.path.normpath(path)) != 'roman-pandeia':
    path = os.path.dirname(path)
repo_path = path
if repo_path not in sys.path:
    sys.path.append(repo_path)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors

from package.plots import overplot
from package.lenses.sample_skypy_lens import SampleSkyPyLens

plt.style.use(f'{repo_path}/package/mplstyle/science.mplstyle')

In [ ]:
figure_dir = os.path.join(repo_path, 'figures')

In [ ]:
pandeia = np.load(os.path.join(repo_path, 'output', 'arrays', 'diagnostics', 'pixel_density_test', f'detector_{oversample_factor}.npy'))

lens = SampleSkyPyLens()
num_pix = 45

model = lens.get_array(num_pix=num_pix)